# UniversalLLM 基本功能测试

测试简化后的UniversalLLM类的基本功能

In [ ]:
# 导入必要的库
import sys
from pathlib import Path

# 添加项目路径
sys.path.append(str(Path.cwd().parent))

from llm_api.universal_llm import create_llm

## 1. 创建LLM实例并查看配置状态

In [ ]:
# 创建LLM实例
llm = create_llm()

# 查看配置状态
print("=== LLM配置状态 ===")
status = llm.get_status()
for key, value in status.items():
    print(f"{key}: {value}")

print(f"\n当前激活的fallback列表: {status['active_fallback_list']}")
print(f"可用的providers: {status['available_providers']}")
print(f"已初始化的clients: {status['initialized_clients']}")

## 2. 基本文本生成测试

In [ ]:
# 简单的文本生成测试
print("=== 基本文本生成测试 ===")

test_prompt = "简单回答：1+1等于几？"

try:
    response = llm.generate(test_prompt, verbose=True)
    print(f"\n✅ 测试成功！")
    print(f"回复: {response}")
    print(f"使用的provider: {llm.current_provider}")
except Exception as e:
    print(f"❌ 测试失败: {e}")

## 3. 中文问答测试

In [ ]:
# 中文问答测试
print("=== 中文问答测试 ===")

chinese_prompt = "用中文简单介绍一下Python编程语言的特点（50字以内）"

try:
    response = llm.generate(chinese_prompt)
    print(f"✅ 中文回复成功")
    print(f"问题: {chinese_prompt}")
    print(f"回复: {response}")
    print(f"使用provider: {llm.current_provider}")
except Exception as e:
    print(f"❌ 中文回复失败: {e}")

## 4. Fallback机制测试

测试当第一个provider不可用时，是否能自动切换到下一个provider

In [ ]:
# 测试fallback机制
print("=== Fallback机制测试 ===")

# 显示当前fallback顺序
print(f"当前fallback顺序: {llm.config.get('active_fallback_list', [])}")

# 进行几次调用，观察是否稳定使用相同provider
for i in range(3):
    try:
        response = llm.generate(f"回答数字：{i+1}")
        print(f"第{i+1}次调用成功，使用provider: {llm.current_provider}")
    except Exception as e:
        print(f"第{i+1}次调用失败: {e}")

## 5. 总结

简化后的UniversalLLM主要特性：

1. **配置简单**: 所有配置都在 `configs/base.yaml` 中
2. **自动fallback**: 按照 `active_fallback_list` 顺序尝试不同provider
3. **支持多种API**: OpenAI兼容API (gpt_free, gpt_light, gpt_pro) 和 Ollama本地API
4. **代码简洁**: 去掉了复杂的配置验证和环境变量fallback逻辑

使用方法：
```python
from llm_api.universal_llm import create_llm

llm = create_llm()
response = llm.generate("你的问题")
```

In [ ]:
import time

# 简单的性能测试
def performance_test(llm, name, prompt="简单回答：1+1等于几？"):
    print(f"\n{name} 性能测试:")
    start_time = time.time()
    
    try:
        response = llm.generate(prompt, verbose=False)
        end_time = time.time()
        
        print(f"耗时: {end_time - start_time:.2f}秒")
        print(f"回复: {response}")
        return True
    except Exception as e:
        end_time = time.time()
        print(f"失败 (耗时: {end_time - start_time:.2f}秒): {e}")
        return False

# 测试可用的LLM
performance_test(ollama_llm, "Ollama")

if openai_llm:
    performance_test(openai_llm, "OpenAI")